In [6]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages',
                      '../src']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

In [107]:
import pickle
import networkx as nx
import pylab as plt
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph

In [108]:
def replace_attribute_key(G, tnode, old_key, new_key):
    value = G.node[tnode][old_key]
    G.node[tnode].pop(old_key, None)
    G.node[tnode][new_key] = value 
G = pickle.load(open(OUTPUT_PATH / 'dependency_graph.p', 'rb'))
attr = G.node.data()._nodes
for k in attr.keys():
    replace_attribute_key(G, k, 'name', '_name')
nx.nx_pydot.write_dot(G, open(OUTPUT_PATH / 'dependency.dot', 'w'))

In [166]:
from rdp import rdp

In [4]:
import geojson
from shapely.geometry import shape
import pandas as pd
from pathlib import Path


def convert(path):
    converted = []
    for p in Path(path).glob('*.GeoJson'):
        d = geojson.load(open(p, 'r'))
        converted.append(dict(
            country_name=d['properties']['name'],
            country_iso=d['properties']['alltags']['ISO3166-1'],
            region_slug='_'.join(['country'] + d['properties']['name'].lower().split(' ')),
            region_name=d['properties']['name'],
            region_type='country',
            dashboard='TRUE',
            population=d['properties']['alltags'].get('population'),
            timezone=d['properties']['alltags'].get('timezone'),
            region_shapefile_wkt_1=None,
            region_shapefile_wkt=shape(d['geometry']).simplify(0.05, preserve_topology=False).wkt
        ))
    pd.DataFrame(converted)[['country_name',
                        'country_iso',
                        'region_slug',
                        'region_name',
                        'region_type',
                        'dashboard',
                        'population',
                        'timezone',
                        'region_shapefile_wkt_1',
                        'region_shapefile_wkt']].to_csv(path / 'converted.csv', index=False)

In [5]:
a = convert(RAW_PATH / 'countries-boundaries')

In [178]:
converted

In [9]:
import pandas as pd
url = 'https://docs.google.com/spreadsheets/d/1xBDV5Zm7zSRPK2TGBoVSHhkvPeqRT9hZFiD4AueoVeI/export?format=csv&id'
df = pd.read_csv(url)

In [14]:
ar = df.query('country_name == "Argentina"').query('region_type == "city"')

In [25]:
ar['day'].unique()

array([19, 12, 13, 20, 24, 17, 10, 21, 14, 25, 11, 18,  9, 23, 16, 15, 22,
        8])

In [23]:
ar.pivot_table(columns='region_slug', index='dow', values='expected_2020').iplot()

In [33]:
i = 8

In [48]:
ar.query('region_slug == "cordoba"')[['region_shapefile_wkt']].iloc[0].values

array(['Polygon ((-64.30875060953628974 -31.27541800853970955, -64.292083943961984 -31.27541800853970955, -64.292083943961984 -31.28375134244235056, -64.27541727838767827 -31.28375134244235056, -64.27541727838767827 -31.29208467634500579, -64.26708394560051829 -31.29208467634500579, -64.26708394560051829 -31.30041801024764681, -64.25875061281337253 -31.30041801024764681, -64.25875061281337253 -31.30875134415030203, -64.24208394723905258 -31.30875134415030203, -64.24208394723905258 -31.31708467805294305, -64.23375061445190681 -31.31708467805294305, -64.23375061445190681 -31.30875134415030203, -64.22541728166476105 -31.30875134415030203, -64.22541728166476105 -31.30041801024764681, -64.21708394887760107 -31.30041801024764681, -64.21708394887760107 -31.30875134415030203, -64.2087506160904411 -31.30875134415030203, -64.2087506160904411 -31.31708467805294305, -64.20041728330329533 -31.31708467805294305, -64.20041728330329533 -31.32541801195559827, -64.19208395051613536 -31.32541801195559827

In [46]:
ar.query('region_slug == "cordoba"')[['observed', 'expected_2020', 'day', 'ratio_20']].sort_values(by='observed')

,observed,expected_2020,day,ratio_20
3835,275761,992141.0,24,0.277945
2403,313484,1161810.0,23,0.269824
3807,501755,463773.0,8,1.081898
3837,631282,992141.0,17,0.636283
4567,699623,571116.0,21,1.225010
337,797922,1573230.0,20,0.507187
4604,984402,1692746.0,19,0.581541
3836,1108259,992141.0,10,1.117038
2404,1257943,1161810.0,9,1.082744
1103,1390271,1104857.0,18,1.258327


In [34]:
ar.query(f'day in ({list(range(i, i+7))})')\
    .pivot_table(columns='region_slug', index='dow', values='observed').iplot()

In [42]:
i = 8+7
ar.query(f'day in ({list(range(i, i+7))})')\
    .pivot_table(columns='region_slug', index='dow', values='observed').iplot()

In [22]:
ar.pivot_table(columns='region_slug', index='dow', values='observed').iplot(kind='bar')

In [49]:
from src import utils
from datetime import datetime
conn = utils.connect_athena(path='../configs/athena.yaml')

In [50]:
df = pd.read_sql_query("""
select 
 *
from spd_sdv_waze_corona.prod_daily_daily_raw
where st_intersects(
  st_polygon('Polygon ((-64.30875060953628974 -31.27541800853970955, -64.292083943961984 -31.27541800853970955, -64.292083943961984 -31.28375134244235056, -64.27541727838767827 -31.28375134244235056, -64.27541727838767827 -31.29208467634500579, -64.26708394560051829 -31.29208467634500579, -64.26708394560051829 -31.30041801024764681, -64.25875061281337253 -31.30041801024764681, -64.25875061281337253 -31.30875134415030203, -64.24208394723905258 -31.30875134415030203, -64.24208394723905258 -31.31708467805294305, -64.23375061445190681 -31.31708467805294305, -64.23375061445190681 -31.30875134415030203, -64.22541728166476105 -31.30875134415030203, -64.22541728166476105 -31.30041801024764681, -64.21708394887760107 -31.30041801024764681, -64.21708394887760107 -31.30875134415030203, -64.2087506160904411 -31.30875134415030203, -64.2087506160904411 -31.31708467805294305, -64.20041728330329533 -31.31708467805294305, -64.20041728330329533 -31.32541801195559827, -64.19208395051613536 -31.32541801195559827, -64.19208395051613536 -31.30875134415030203, -64.18375061772897539 -31.30875134415030203, -64.18375061772897539 -31.30041801024764681, -64.16708395215468386 -31.30041801024764681, -64.16708395215468386 -31.30875134415030203, -64.15875061936752388 -31.30875134415030203, -64.15875061936752388 -31.34208467976088031, -64.14208395379321814 -31.34208467976088031, -64.14208395379321814 -31.35041801366353553, -64.12541728821889819 -31.35041801366353553, -64.12541728821889819 -31.35875134756617655, -64.11708395543175243 -31.35875134756617655, -64.11708395543175243 -31.36708468146883177, -64.10875062264460666 -31.36708468146883177, -64.10875062264460666 -31.37541801537147279, -64.10041728985744669 -31.37541801537147279, -64.10041728985744669 -31.38375134927412802, -64.09208395707028671 -31.38375134927412802, -64.09208395707028671 -31.39208468317676903, -64.10041728985744669 -31.39208468317676903, -64.10041728985744669 -31.40041801707942426, -64.10875062264460666 -31.40041801707942426, -64.10875062264460666 -31.41708468488470629, -64.10041728985744669 -31.41708468488470629, -64.10041728985744669 -31.44208468659265776, -64.09208395707028671 -31.44208468659265776, -64.09208395707028671 -31.45041802049529878, -64.08375062428314095 -31.45041802049529878, -64.08375062428314095 -31.458751354397954, -64.07541729149599519 -31.458751354397954, -64.07541729149599519 -31.46708468830059502, -64.08375062428314095 -31.46708468830059502, -64.08375062428314095 -31.47541802220323603, -64.12541728821889819 -31.47541802220323603, -64.12541728821889819 -31.48375135610589126, -64.13375062100605817 -31.48375135610589126, -64.13375062100605817 -31.49208469000853228, -64.15875061936752388 -31.49208469000853228, -64.15875061936752388 -31.48375135610589126, -64.19208395051613536 -31.48375135610589126, -64.19208395051613536 -31.49208469000853228, -64.20041728330329533 -31.49208469000853228, -64.20041728330329533 -31.5004180239111875, -64.21708394887760107 -31.5004180239111875, -64.21708394887760107 -31.49208469000853228, -64.22541728166476105 -31.49208469000853228, -64.22541728166476105 -31.5004180239111875, -64.25041728002621255 -31.5004180239111875, -64.25041728002621255 -31.49208469000853228, -64.25875061281337253 -31.49208469000853228, -64.25875061281337253 -31.48375135610589126, -64.26708394560051829 -31.48375135610589126, -64.26708394560051829 -31.458751354397954, -64.27541727838767827 -31.458751354397954, -64.27541727838767827 -31.45041802049529878, -64.28375061117483824 -31.45041802049529878, -64.28375061117483824 -31.42541801878736152, -64.27541727838767827 -31.42541801878736152, -64.27541727838767827 -31.41708468488470629, -64.28375061117483824 -31.41708468488470629, -64.28375061117483824 -31.39208468317676903, -64.292083943961984 -31.39208468317676903, -64.292083943961984 -31.37541801537147279, -64.30041727674912977 -31.37541801537147279, -64.30041727674912977 -31.36708468146883177, -64.292083943961984 -31.36708468146883177, -64.292083943961984 -31.35875134756617655, -64.30041727674912977 -31.35875134756617655, -64.30041727674912977 -31.35041801366353553, -64.30875060953628974 -31.35041801366353553, -64.30875060953628974 -31.32541801195559827, -64.31708394232344972 -31.32541801195559827, -64.31708394232344972 -31.31708467805294305, -64.32541727511059548 -31.31708467805294305, -64.32541727511059548 -31.30875134415030203, -64.31708394232344972 -31.30875134415030203, -64.31708394232344972 -31.28375134244235056, -64.30875060953628974 -31.28375134244235056, -64.30875060953628974 -31.27541800853970955),(-64.23375061445190681 -31.31708467805294305, -64.23375061445190681 -31.32541801195559827, -64.22541728166476105 -31.32541801195559827, -64.22541728166476105 -31.31708467805294305, -64.23375061445190681 -31.31708467805294305))'),
      st_line(line))""", conn)

In [59]:
day = df[ df['retrievaltime'].apply(lambda x: (x.day == 11) or (x.day == 12))]

In [61]:
df['day'] = df['retrievaltime'].apply(lambda x: x.day)

In [66]:
df.groupby('day').sum()['length'].iplot()

In [69]:
df.groupby('day').count()['length'].iplot(
    title='Number of Records for Cordoba per day in March 2020',
    theme='custom'
)

In [92]:
counta = pd.read_sql_query(
"""
with t as (
    select
    year(retrievaltime) as year,
    month(retrievaltime) as month,
    day(retrievaltime) as day,
    day_of_week(retrievaltime) as dow,
    date_parse(format_datetime(date_add('minute', 
                                cast(date_diff('minute',
                                    timestamp '2015-01-01 00:00:00', retrievaltime) / 5 as bigint) * 5,
                                    timestamp '2015-01-01 00:00:00'), 'H:m'), '%H:%i') as time,
    row_number() over (partition by uuid, 
                            date_parse(format_datetime(date_add('minute', 
                                cast(date_diff('minute',
                                    timestamp '2015-01-01 00:00:00', retrievaltime) / 5 as bigint) * 5,
                                  timestamp '2015-01-01 00:00:00'), 'H:m'), '%H:%i') order by retrievaltime) n_row
    from (
        select 
            uuid, arbitrary(from_unixtime(retrievaltime/1000)) retrievaltime
        from "p-waze-parquet-waze"."jams"
        where regexp_like(datetime, '202002')
        and
            speed >= 0
        and length > 0
        group by
            uuid, pubmillis, country, 
                    city, street, roadtype, level, length, speed,
                    speedkmh, delay, line,  type,  turntype,
                    blockingalertuuid, startnode, endnode))
select 
    month,
    day,
    count(*)
from t
where n_row = 1
group by month, day
""", conn)

In [91]:
counta.set_index(['month', 'day']).sort_index().iplot(
    title='Deduplicated every 5 minutes Number of Records March 2020',
    theme='custom'
)

In [105]:
counta = pd.read_sql_query(
"""
select 
month(from_unixtime(retrievaltime/1000)) month,
day(from_unixtime(retrievaltime/1000)) day,
count(*) as counta
from "p-waze-parquet-waze"."jams"
where regexp_like(datetime, '202002|202003')
and
	speed >= 0
and length > 0	
group by 
month(from_unixtime(retrievaltime/1000)),
day(from_unixtime(retrievaltime/1000))

""", conn)

In [106]:
counta.set_index(['month', 'day']).sort_index().iplot(
    title='Raw Data Number of Records in Feb and March 2020',
    theme='custom'
)

In [102]:
dupls = pd.read_sql_query(
"""
select 
month,
day,
count(*) as counta
from (
    select * 
    from "p-waze-parquet-waze"."jams"
    where regexp_like(datetime, '202002|202003')
    and speed >= 0
    and length > 0
    group by retrievaltime, uuid, pubmillis, country, 
            city, street, roadtype, level, length, speed,
            speedkmh, delay, line,  type,  turntype,
            blockingalertuuid, startnode, endnode, day, month, year, hour, datetime)
group by 
month,
day

""", conn)

In [103]:
dupls.set_index(['month', 'day']).sort_index().iplot(
    title='Full deduplication Raw Data Number of Records in Feb and March 2020',
    theme='custom'
)

In [114]:
a = dupls.merge(counta, on=['month', 'day'])
a['diff'] = a['counta_x'] - a['counta_y']
a.set_index(['month', 'day']).sort_index()['diff'].iplot(
    title='Difference of Raw Data and Deduplicated Number of Records',
    theme='custom'
)

In [118]:
json = pd.read_sql_query(
"""
select 
month,
day,
count(*) as counta 
from "p-waze-json-waze"."jams"
where (month = '3') and year = '2020'
and day in ('1', '2', '3', '4', '5')
group by 
month,
day
""", conn)

In [120]:
json.set_index(['month', 'day']).sort_index().iplot(
    title='JSON number of files',
    theme='custom',
    xTitle='(month, day)'
)

In [121]:
cordoba_count =  pd.read_sql_query(
"""
select 
month(from_unixtime(retrievaltime/1000)) month,
day(from_unixtime(retrievaltime/1000)) day,
count(*) as counta
from "p-waze-parquet-waze"."jams"
where regexp_like(datetime, '202002|202003')
and
	speed >= 0
and length > 0
and st_intersects(
  st_polygon('Polygon ((-64.30875060953628974 -31.27541800853970955, -64.292083943961984 -31.27541800853970955, -64.292083943961984 -31.28375134244235056, -64.27541727838767827 -31.28375134244235056, -64.27541727838767827 -31.29208467634500579, -64.26708394560051829 -31.29208467634500579, -64.26708394560051829 -31.30041801024764681, -64.25875061281337253 -31.30041801024764681, -64.25875061281337253 -31.30875134415030203, -64.24208394723905258 -31.30875134415030203, -64.24208394723905258 -31.31708467805294305, -64.23375061445190681 -31.31708467805294305, -64.23375061445190681 -31.30875134415030203, -64.22541728166476105 -31.30875134415030203, -64.22541728166476105 -31.30041801024764681, -64.21708394887760107 -31.30041801024764681, -64.21708394887760107 -31.30875134415030203, -64.2087506160904411 -31.30875134415030203, -64.2087506160904411 -31.31708467805294305, -64.20041728330329533 -31.31708467805294305, -64.20041728330329533 -31.32541801195559827, -64.19208395051613536 -31.32541801195559827, -64.19208395051613536 -31.30875134415030203, -64.18375061772897539 -31.30875134415030203, -64.18375061772897539 -31.30041801024764681, -64.16708395215468386 -31.30041801024764681, -64.16708395215468386 -31.30875134415030203, -64.15875061936752388 -31.30875134415030203, -64.15875061936752388 -31.34208467976088031, -64.14208395379321814 -31.34208467976088031, -64.14208395379321814 -31.35041801366353553, -64.12541728821889819 -31.35041801366353553, -64.12541728821889819 -31.35875134756617655, -64.11708395543175243 -31.35875134756617655, -64.11708395543175243 -31.36708468146883177, -64.10875062264460666 -31.36708468146883177, -64.10875062264460666 -31.37541801537147279, -64.10041728985744669 -31.37541801537147279, -64.10041728985744669 -31.38375134927412802, -64.09208395707028671 -31.38375134927412802, -64.09208395707028671 -31.39208468317676903, -64.10041728985744669 -31.39208468317676903, -64.10041728985744669 -31.40041801707942426, -64.10875062264460666 -31.40041801707942426, -64.10875062264460666 -31.41708468488470629, -64.10041728985744669 -31.41708468488470629, -64.10041728985744669 -31.44208468659265776, -64.09208395707028671 -31.44208468659265776, -64.09208395707028671 -31.45041802049529878, -64.08375062428314095 -31.45041802049529878, -64.08375062428314095 -31.458751354397954, -64.07541729149599519 -31.458751354397954, -64.07541729149599519 -31.46708468830059502, -64.08375062428314095 -31.46708468830059502, -64.08375062428314095 -31.47541802220323603, -64.12541728821889819 -31.47541802220323603, -64.12541728821889819 -31.48375135610589126, -64.13375062100605817 -31.48375135610589126, -64.13375062100605817 -31.49208469000853228, -64.15875061936752388 -31.49208469000853228, -64.15875061936752388 -31.48375135610589126, -64.19208395051613536 -31.48375135610589126, -64.19208395051613536 -31.49208469000853228, -64.20041728330329533 -31.49208469000853228, -64.20041728330329533 -31.5004180239111875, -64.21708394887760107 -31.5004180239111875, -64.21708394887760107 -31.49208469000853228, -64.22541728166476105 -31.49208469000853228, -64.22541728166476105 -31.5004180239111875, -64.25041728002621255 -31.5004180239111875, -64.25041728002621255 -31.49208469000853228, -64.25875061281337253 -31.49208469000853228, -64.25875061281337253 -31.48375135610589126, -64.26708394560051829 -31.48375135610589126, -64.26708394560051829 -31.458751354397954, -64.27541727838767827 -31.458751354397954, -64.27541727838767827 -31.45041802049529878, -64.28375061117483824 -31.45041802049529878, -64.28375061117483824 -31.42541801878736152, -64.27541727838767827 -31.42541801878736152, -64.27541727838767827 -31.41708468488470629, -64.28375061117483824 -31.41708468488470629, -64.28375061117483824 -31.39208468317676903, -64.292083943961984 -31.39208468317676903, -64.292083943961984 -31.37541801537147279, -64.30041727674912977 -31.37541801537147279, -64.30041727674912977 -31.36708468146883177, -64.292083943961984 -31.36708468146883177, -64.292083943961984 -31.35875134756617655, -64.30041727674912977 -31.35875134756617655, -64.30041727674912977 -31.35041801366353553, -64.30875060953628974 -31.35041801366353553, -64.30875060953628974 -31.32541801195559827, -64.31708394232344972 -31.32541801195559827, -64.31708394232344972 -31.31708467805294305, -64.32541727511059548 -31.31708467805294305, -64.32541727511059548 -31.30875134415030203, -64.31708394232344972 -31.30875134415030203, -64.31708394232344972 -31.28375134244235056, -64.30875060953628974 -31.28375134244235056, -64.30875060953628974 -31.27541800853970955),(-64.23375061445190681 -31.31708467805294305, -64.23375061445190681 -31.32541801195559827, -64.22541728166476105 -31.32541801195559827, -64.22541728166476105 -31.31708467805294305, -64.23375061445190681 -31.31708467805294305))'),
      st_line(line))
group by 
month(from_unixtime(retrievaltime/1000)),
day(from_unixtime(retrievaltime/1000))
""", conn)

In [123]:
duplsdupls = pd.read_sql_query(
"""
select 
month,
day,
count(*) as counta
from (
    select uuid, pubmillis, country, 
            city, street, roadtype, level, length, speed,
            speedkmh, delay, line,  type,  turntype,
            blockingalertuuid, startnode, endnode, day, month, year, hour, datetime 
    from "p-waze-parquet-waze"."jams"
    where regexp_like(datetime, '202002|202003')
    and speed >= 0
    and length > 0
    group by uuid, pubmillis, country, 
            city, street, roadtype, level, length, speed,
            speedkmh, delay, line,  type,  turntype,
            blockingalertuuid, startnode, endnode, day, month, year, hour, datetime)
group by 
month,
day

""", conn)

In [124]:
duplsdupls.set_index(['month', 'day']).sort_index().iplot(
    title='JSON number of files',
    theme='custom',
    xTitle='(month, day)'
)

In [ ]:
json = pd.read_sql_query(
"""
select 
month(from_unixtime(retrievaltime/1000)) month,
day(from_unixtime(retrievaltime/1000)) day,
count(*) as counta 
from spd_sdv_waze_reprocessed.jams_ready
where regexp_like(datetime, '202002|202003')
group by 
month(from_unixtime(retrievaltime/1000)),
day(from_unixtime(retrievaltime/1000))
""", conn)

In [ ]:
json.set_index(['month', 'day']).sort_index().iplot(
    title='JSON number of files',
    theme='custom',
    xTitle='(month, day)'
)

In [60]:
day.to_csv(OUTPUT_PATH / 'cordoba_test.csv')